                                Proyecto Electrico

                                Escuela de Ingenieria Electrica

                                LabCes

                                Titulo: Implementacion de Analog Ensembles para prediccion de irradiancia solar.

                                Estudiante: Elsa Valeria Roman Astua
                                    
                                Carnet: C26910

                                Profesor: Fausto Calderon Obaldia

                                Version: 02

                                Descripción: Este código procesa
                               datos históricos de irradiancia solar,
                               entrena un modelo basado en Analog
                               Ensemble (AnEn) y permite generar
                               pronósticos interactivos. Construye
                               ventanas temporales, estandariza
                               predictores, busca análogos en el
                               historial y produce un pronóstico con
                               incertidumbre, además de mostrar una
                               gráfica de la ventana temporal
                               alrededor del tiempo predicho.



In [1]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
from datetime import datetime, timedelta
import sys
import matplotlib.pyplot as plt

# Montar Google Drive
drive.mount('/content/drive')

# Configuración general
INPUT_FOLDER = '/content/drive/MyDrive/Datos Filtrados/'
TIME_COLUMN = 'timestamp'
TARGET_COLUMN = 'solar'
L = 20  # Pasos del pronóstico
RESAMPLE_FREQ_MINUTES = 3
L_minutes = L * RESAMPLE_FREQ_MINUTES
PREDICTOR_COLUMNS = [
    'windspeed', 'wind_direction', 'avg_temperature',
    'relative_humidity', 'barometric_pressure', TARGET_COLUMN
]

WINDOW_HALF_HOURS = 4
WINDOW_STEPS = WINDOW_HALF_HOURS * (60 // RESAMPLE_FREQ_MINUTES)

# Variables globales del modelo
F_hist_np = None
O_hist_np = None
predictor_names = PREDICTOR_COLUMNS
mu = None
sigma = None
F_hist_standardized = None

# Solicitud de entrada en Colab
def get_user_input(prompt_text):
    return input(prompt_text).strip()

# Construcción de F_hist y O_hist con ventana temporal
def combine_and_structure_data_windowed(folder_path, predictor_cols, target_col, L):
    global F_hist_np, O_hist_np, predictor_names

    all_data = []
    print("\nESTRUCTURANDO DATOS")

    for filename in os.listdir(folder_path):
        if filename.endswith("_filtrado.csv"):
            df = pd.read_csv(os.path.join(folder_path, filename),
                             index_col=TIME_COLUMN, parse_dates=True)
            all_data.append(df)

    if not all_data:
        raise FileNotFoundError("No se encontraron archivos válidos.")

    df_historia = pd.concat(all_data).sort_index()
    df_historia = df_historia[~df_historia.index.duplicated()]
    df_historia = df_historia.dropna(subset=predictor_cols + [target_col])

    N = len(df_historia)
    window_length = 2 * WINDOW_STEPS + 1

    O_list = []
    start_index_F = WINDOW_STEPS
    end_index_F = N - (L + WINDOW_STEPS)

    for i in range(start_index_F, end_index_F):
        window_start = i - WINDOW_STEPS
        window_end = i + L + WINDOW_STEPS + 1
        O_list.append(df_historia[target_col].iloc[window_start:window_end].values)

    O_hist_np = np.array(O_list)
    F_hist = df_historia[predictor_cols].iloc[start_index_F:end_index_F]

    # Ajuste de tamaños
    if len(F_hist) != len(O_hist_np):
        m = min(len(F_hist), len(O_hist_np))
        F_hist = F_hist.iloc[:m]
        O_hist_np = O_hist_np[:m]

    F_hist_np = F_hist.values
    predictor_names = predictor_cols

    print(f"Pares generados: {len(F_hist_np)}")
    return F_hist_np, O_hist_np, predictor_cols

# Estandarización del modelo
def train_model_standardize():
    global mu, sigma, F_hist_standardized

    if F_hist_np is None:
        raise ValueError("F_hist_np no está definido.")

    mu = np.mean(F_hist_np, axis=0)
    sigma = np.std(F_hist_np, axis=0)
    sigma[sigma == 0] = 1.0

    F_hist_standardized = (F_hist_np - mu) / sigma
    print("Estandarización completada.")

# Predicción ANEN
def make_prediction_anen(F_t_nuevo: np.ndarray, M: int = 75) -> dict:
    F_t_standardized = (F_t_nuevo - mu) / sigma
    distances = np.sqrt(np.sum((F_t_standardized - F_hist_standardized)**2, axis=1))
    analog_indices = np.argsort(distances)[:M]

    analog_ensemble = O_hist_np[analog_indices]
    prediction_point_index = WINDOW_STEPS

    mean_fc = np.mean(analog_ensemble, axis=0)[prediction_point_index]
    q05 = np.percentile(analog_ensemble[:, prediction_point_index], 5)
    q95 = np.percentile(analog_ensemble[:, prediction_point_index], 95)

    return {
        "ensemble_size": M,
        "mean_forecast": mean_fc,
        "median_forecast": np.median(analog_ensemble[:, prediction_point_index]),
        "uncertainty_range": (q05, q95),
        "ensemble_members": analog_ensemble
    }

# Gráfica de la ventana
def plot_prediction_window(analog_ensemble: np.ndarray, target_datetime: datetime):
    mean = np.mean(analog_ensemble, axis=0)
    q05 = np.percentile(analog_ensemble, 5, axis=0)
    q95 = np.percentile(analog_ensemble, 95, axis=0)

    start = target_datetime - timedelta(minutes=WINDOW_STEPS * RESAMPLE_FREQ_MINUTES)
    time_index = pd.date_range(start=start, periods=len(mean),
                               freq=f'{RESAMPLE_FREQ_MINUTES}min')

    prediction_idx = WINDOW_STEPS
    plt.figure(figsize=(12, 6))
    plt.fill_between(time_index, q05, q95, color='orange', alpha=0.3)
    plt.plot(time_index, mean, color='blue')
    plt.scatter(target_datetime, mean[prediction_idx], color='red')
    plt.axvline(target_datetime, color='red', linestyle='--')
    plt.title('Pronóstico ANEN con Ventana')
    plt.xlabel('Tiempo'); plt.ylabel('Irradiancia (W/m²)')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# Ejecución interactiva
def run_interactive_prediction_windowed(predictor_cols):
    if F_hist_np is None or mu is None:
        print("El modelo no está entrenado.")
        return

    date_str = get_user_input("Fecha (YYYY-MM-DD): ")
    time_str = get_user_input("Hora (HH:MM): ")

    try:
        target_datetime = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
        input_datetime = target_datetime - timedelta(minutes=L_minutes)
    except ValueError:
        print("Formato inválido.")
        return

    print(f"\nIngrese valores correspondientes a {input_datetime}:")

    F_t_data = []
    for col in predictor_cols:
        while True:
            try:
                F_t_data.append(float(get_user_input(f"{col}: ")))
                break
            except ValueError:
                print("Ingrese un número válido.")

    prediction = make_prediction_anen(np.array(F_t_data), M=75)

    plot_prediction_window(prediction['ensemble_members'], target_datetime)

    q05, q95 = prediction["uncertainty_range"]
    print(f"\nMedia: {prediction['mean_forecast']:.2f}")
    print(f"Mediana: {prediction['median_forecast']:.2f}")
    print(f"Rango 90%: {q05:.2f} - {q95:.2f}")

# Orquestación
try:
    F_hist_np, O_hist_np, predictor_names = combine_and_structure_data_windowed(
        INPUT_FOLDER, PREDICTOR_COLUMNS, TARGET_COLUMN, L
    )
    train_model_standardize()
    run_interactive_prediction_windowed(predictor_names)

except Exception as e:
    print(f"Error en la ejecución: {e}")


Mounted at /content/drive

ESTRUCTURANDO DATOS
Pares generados: 109687
Estandarización completada.


KeyboardInterrupt: Interrupted by user